In [1]:
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bokeh.models import CustomJS
from bokeh.io import output_file, output_notebook
from bokeh.layouts import widgetbox, layout, row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, NumberFormatter, Select
from bokeh.plotting import Figure, output_file, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
import lsst.sims.maf.metricBundles as mb
import lsst.sims.maf.plots as plots
import lsst.sims.maf.runComparison as rc

In [10]:
def generateDiffHtml(comp_object, normalized = False, html_out = None, show_page = False):
    """
    Use `bokeh` to convert the dataframe returned by `summaryDiffReport` into
    an interactive html table.
    Parameters
    ----------
    dataframe : pandas DataFrame
        Dataframe containg the summary stats of multpile opsim runs.
    html_out : str
        Name of the html that will be output and saved.
    show_page : bool, opt
        If True the html page generate by this function will automatically open
        in your browser
    """
    if html_out is not None:
        output_file(html_out, title = html_out.strip('.html'))
    
    if normalized is False:
        dataframe = comp_object.headerStats.T.merge(comp.summaryStats.T, left_index=True, right_index=True)
    else:
        dataframe = comp_object.headerStats.T.merge(comp.normalizedStats.T, left_index=True, right_index=True)
   
    dataframe.reset_index(level=0, inplace=True)
    dataframe.columns.values[0]='FullName' 

    columns = []

    for col in dataframe.columns:

        if col not in ['FullName','BaseName','SummaryType']:
            columns.append(TableColumn(field=col, title=col, formatter=NumberFormatter(format="0.0000")))
        else:
            columns.append(TableColumn(field=col, title=col))
    source = ColumnDataSource(dataframe)
    original_source = ColumnDataSource(dataframe)
    data_table = DataTable(source=source, columns=columns, width=1700, height=2000)

    combined_callback_code = """
    var data = source.data;
    var original_data = original_source.data;
    var FullName= FullName_select_obj.value;
    var BaseName = BaseName_select_obj.value;
    var SummaryType = SummaryType_select_obj.value;
     for (var key in original_data) {
         data[key] = [];
         for (var i = 0; i < original_data['FullName'].length; ++i) {
             if ((FullName === "ALL" || original_data['FullName'][i] === FullName) &&
                 (BaseName === "ALL" || original_data['BaseName'][i] === BaseName) &&
                 (SummaryType === "ALL" || original_data['SummaryType'][i] === SummaryType)) {
                 data[key].push(original_data[key][i]);
             }
         }
     }
    source.change.emit();
    target_obj.change.emit();
    """

    FullName_list = ['ALL'] + dataframe['FullName'].unique().tolist()
    FullName_select = Select(title="FullName:", value=FullName_list[0], options=FullName_list)

    BaseName_list = ['ALL'] + dataframe['BaseName'].unique().tolist()
    BaseName_select = Select(title="BaseName:",
                                   value=BaseName_list[0],
                                   options=BaseName_list)

    SummaryType_list = ['ALL'] + dataframe['SummaryType'].unique().tolist()
    SummaryType_select = Select(title="SummaryType:",
                                value=SummaryType_list[0],
                                options=SummaryType_list)


    generic_callback = CustomJS(args=dict(source=source,
                                          original_source=original_source,
                                          FullName_select_obj=FullName_select,
                                          BaseName_select_obj=BaseName_select,
                                          SummaryType_select_obj=SummaryType_select,
                                          target_obj=data_table),
                                code=combined_callback_code)

    FullName_select.callback = generic_callback
    BaseName_select.callback = generic_callback
    SummaryType_select.callback = generic_callback



    dropdownMenus = column([FullName_select, BaseName_select, SummaryType_select])
    page_layout = layout([dropdownMenus,data_table])
    show(page_layout)

In [5]:
runlist = ['colossus_2218',
           'colossus_2210',
           'colossus_2211',
           'colossus_2212']

In [6]:
comp = rc.RunComparison(baseDir='.', runlist=runlist, verbose=True)


In [7]:
for r in comp.runlist:
    print(r, comp.runresults[r].keys())

colossus_2218 dict_keys(['sci', 'sched', 'cadence'])
colossus_2210 dict_keys(['sci', 'sched', 'cadence'])
colossus_2211 dict_keys(['sci', 'sched', 'cadence'])
colossus_2212 dict_keys(['sci', 'sched', 'cadence'])


In [8]:
# Read in all summary metrics, from all of these analysis.
mdicts = comp.buildMetricDict(subdir='sci')
# Suppress warnings from metrics which do not have summary stats. 
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    comp.addSummaryStats(mdicts)

In [11]:
generateDiffHtml(comp)

In [12]:
comp.normalizeStats(baselineRun='colossus_2218')

In [13]:
generateDiffHtml(comp,normalized=True)